In [1]:
import csv
import pika

# ... code for connecting to RabbitMQ ...
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Declare the queue
channel.queue_declare(queue='csv_data')


with open('RTPData.csv', 'r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        # Access the file path and print it
        print(f"Sending record: {file.name} - {row}")
        channel.basic_publish(exchange='',
                              routing_key='csv_data',
                              body=','.join(row))

connection.close()


Sending record: live4.csv - ['0', '1398', '105017', '3', '13311.36989', '39653', '105017', '369', '105023', '330', '369', '105023', '345.6', '432', '349.5', '-1', '349.5', '330', '17', '0']
Sending record: live4.csv - ['0', '750', '1', '1', '750000000', '34181', '1', '375', '1', '375', '375', '1', '375', '562.5', '375', '-1', '375', '375', '17', '0']
Sending record: live4.csv - ['0', '0', '99', '0', '0', '12187', '1', '0', '101', '0', '0', '1', '0', '0', '0', '5840', '0', '0', '6', '1']
Sending record: live4.csv - ['0', '802', '1', '1', '802000000', '50534', '1', '401', '1', '401', '401', '1', '401', '601.5', '401', '-1', '401', '401', '17', '0']
Sending record: live4.csv - ['0', '2944', '3', '1', '981000000', '672', '3', '1472', '3', '1472', '1472', '3', '1472', '2208', '1472', '-1', '1472', '1472', '17', '0']
Sending record: live4.csv - ['0', '1398', '107748', '3', '12974.47796', '47220', '107748', '369', '107750', '330', '369', '107750', '345.6', '432', '349.5', '-1', '349.5', '330'

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [3]:
# Define preprocess_data function
import joblib

model = joblib.load('modelMNB.pkl') 

def preprocess_data(X):
    # Scale the features using StandardScaler
    scaler = MinMaxScaler()
    scaled_X = scaler.fit_transform(X)
    return scaled_X  # Correct indentation


# Callback function to handle incoming messages
def callback(ch, method, properties, body):
    # Assuming body contains the data for a single row in CSV format
    data = body.decode('utf-8').strip().split(',')  # Assuming CSV data
    # Assuming data is in the same format as the training data
    # Convert data to DataFrame with appropriate column names
    df = pd.DataFrame([data])
    # Assuming you have preprocessed your data similarly to the training data
    # Make predictions
    #print(df)
    prediction = model.predict(df)
    print("Prediction:", prediction)
    


# Connect to RabbitMQ server
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()


# Declare the queue
channel.queue_declare(queue='csv_data')


# Set up consumer to receive messages from the queue
channel.basic_consume(queue='csv_data',
                      on_message_callback=callback,
                      auto_ack=True)

# Start consuming messages
print('Waiting for messages...')

import threading
st = threading.Timer(2,channel.stop_consuming)
st.start()

try:
    channel.start_consuming()
except KeyboardInterrupt:
    channel.stop_consuming()
finally:
    connection.close()
 
print("done!")


Waiting for messages...
Prediction: ['Syn']
Prediction: ['DrDoS_UDP']
Prediction: ['BENIGN']
Prediction: ['DrDoS_UDP']
Prediction: ['DrDoS_UDP']
Prediction: ['Syn']
Prediction: ['Syn']
Prediction: ['DrDoS_UDP']
Prediction: ['BENIGN']
Prediction: ['Syn']
done!
